In [8]:
# Install required packages
!pip install opencv-python pandas numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# Test FFmpeg installation
import subprocess

try:
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
    print("✅ FFmpeg found!")
    print(result.stdout.split('\n')[0])
except FileNotFoundError:
    print("❌ FFmpeg not found!")
    print("\nInstall FFmpeg:")
    print("  Option 1: winget install ffmpeg")
    print("  Option 2: Download from https://ffmpeg.org/download.html")
    print("\nMake sure FFmpeg is in your PATH!")

✅ FFmpeg found!
ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers


In [10]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import subprocess
import os
import time
import json

def extract_features(video_path):
    """Extract 7 features from video for ML training"""
    cap = cv2.VideoCapture(str(video_path))
    
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps if fps > 0 else 0
    
    # Sample 1 frame per second
    sample_interval = int(fps) if fps > 0 else 1
    
    edge_scores = []
    motion_scores = []
    brightness_scores = []
    color_variances = []
    prev_frame = None
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Only process sampled frames
        if frame_idx % sample_interval == 0:
            # Edge density
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 100, 200)
            edge_scores.append(np.mean(edges) / 255.0)
            
            # Motion (frame difference)
            if prev_frame is not None:
                diff = cv2.absdiff(gray, prev_frame)
                motion_scores.append(np.mean(diff) / 255.0)
            prev_frame = gray.copy()
            
            # Brightness
            brightness_scores.append(np.mean(gray) / 255.0)
            
            # Color variance
            color_variances.append(np.std(frame) / 255.0)
        
        frame_idx += 1
    
    cap.release()
    
    return {
        'edge_density': np.mean(edge_scores) if edge_scores else 0,
        'motion_score': np.mean(motion_scores) if motion_scores else 0,
        'brightness': np.mean(brightness_scores) if brightness_scores else 0,
        'color_variance': np.mean(color_variances) if color_variances else 0,
        'resolution': width * height,
        'fps': fps,
        'duration': duration
    }

print("✅ Feature extraction ready")

✅ Feature extraction ready


In [11]:
def encode_video(input_path, output_path, crf, preset):
    """Encode video using CPU (libx264)"""
    cmd = [
        'ffmpeg', '-y', '-hide_banner', '-loglevel', 'error',
        '-i', str(input_path),
        '-c:v', 'libx264',
        '-preset', preset,
        '-crf', str(crf),
        '-c:a', 'aac', '-b:a', '128k',
        str(output_path)
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"Encoding failed: {result.stderr}")
    
    return output_path

def measure_psnr(distorted_path, reference_path):
    """Measure quality using SSIM - fast and reliable"""
    import re
    
    # Use SSIM for fast quality measurement (same as terminal test)
    cmd_ssim = [
        'ffmpeg', '-hide_banner',
        '-i', str(distorted_path),
        '-i', str(reference_path),
        '-lavfi', 'ssim',
        '-f', 'null', '-'
    ]
    
    result = subprocess.run(cmd_ssim, capture_output=True, text=True)
    
    # SSIM outputs to stderr: "SSIM Y:0.XXXX U:0.XXXX V:0.XXXX All:0.XXXX"
    match = re.search(r'All:(\d+\.\d+)', result.stderr)
    if match:
        ssim_value = float(match.group(1))
        
        # Same SSIM to quality conversion used in terminal test
        if ssim_value >= 0.95:
            quality_score = 85 + (ssim_value - 0.95) * 300  # 0.95-1.0 maps to 85-100
        elif ssim_value >= 0.90:
            quality_score = 70 + (ssim_value - 0.90) * 300  # 0.90-0.95 maps to 70-85
        elif ssim_value >= 0.85:
            quality_score = 55 + (ssim_value - 0.85) * 300  # 0.85-0.90 maps to 55-70
        else:
            quality_score = max(0, ssim_value * 64.7)  # Below 0.85 maps to 0-55
        
        return quality_score
    
    # Fallback: estimate from file size ratio (should rarely hit this)
    orig_size = get_file_size_mb(reference_path)
    new_size = get_file_size_mb(distorted_path)
    compression_ratio = new_size / orig_size
    quality_score = max(50, min(95, 120 - (compression_ratio * 100)))
    
    return quality_score

def get_file_size_mb(path):
    """Get file size in MB"""
    return os.path.getsize(path) / (1024 * 1024)

print("✅ Encoding and quality measurement ready")

✅ Encoding and quality measurement ready


In [12]:
def find_optimal_settings(video_path, crf_values=[24, 26, 28, 30], presets=['faster', 'fast', 'medium'], quality_threshold=90):
    """Test all CRF/preset combinations and find optimal settings"""
    print(f"\n🎬 Processing: {Path(video_path).name}")
    
    results = []
    temp_dir = Path('temp_encodings')
    temp_dir.mkdir(exist_ok=True)
    
    # Test all combinations
    for crf in crf_values:
        for preset in presets:
            output_path = temp_dir / f"test_crf{crf}_{preset}.mp4"
            
            try:
                # Encode
                start_time = time.time()
                encode_video(video_path, output_path, crf, preset)
                encode_time = time.time() - start_time
                
                # Measure quality
                quality_score = measure_psnr(output_path, video_path)
                file_size = get_file_size_mb(output_path)
                
                results.append({
                    'crf': crf,
                    'preset': preset,
                    'quality': quality_score,
                    'size_mb': file_size,
                    'encode_time': encode_time
                })
                
                print(f"  CRF {crf} + {preset:7s}: Quality {quality_score:.1f}, {file_size:.1f}MB, {encode_time:.1f}s")
                
                # Clean up temp file
                output_path.unlink()
                
            except Exception as e:
                print(f"  ❌ CRF {crf} + {preset}: Failed - {str(e)[:100]}")
    
    # Find optimal: smallest file where quality >= threshold
    valid_results = [r for r in results if r['quality'] >= quality_threshold]
    
    if not valid_results:
        if not results:
            raise RuntimeError("All encoding attempts failed")
        # No results meet threshold, pick best quality
        best = max(results, key=lambda x: x['quality'])
        print(f"  ⚠️  No settings meet quality {quality_threshold}, using best: CRF {best['crf']} + {best['preset']}")
        return best['crf'], best['preset'], best['quality'], best['size_mb']
    
    # Pick smallest file from valid results
    best = min(valid_results, key=lambda x: x['size_mb'])
    print(f"  ✅ Optimal: CRF {best['crf']} + {best['preset']} (Quality {best['quality']:.1f}, {best['size_mb']:.1f}MB)")
    
    return best['crf'], best['preset'], best['quality'], best['size_mb']

print("✅ Optimization function ready")

✅ Optimization function ready


## Configure Your Videos

Put your MP4 videos in a folder and update the path below:

In [13]:
# UPDATE THIS PATH to your video folder
video_dir = Path(r'training_videos/')  # Change this!

# Create test folder if it doesn't exist
video_dir.mkdir(exist_ok=True)

# Get list of video files
video_files = list(video_dir.glob('*.mp4')) + list(video_dir.glob('*.avi')) + list(video_dir.glob('*.mkv'))
print(f"Found {len(video_files)} videos")

if len(video_files) == 0:
    print(f"\n⚠️  No videos found in: {video_dir}")
    print("\nPlease:")
    print("  1. Put some MP4 files in the folder above")
    print("  2. Or update the video_dir path to your video folder")
else:
    for v in video_files[:10]:
        print(f"  - {v.name}")
    if len(video_files) > 10:
        print(f"  ... and {len(video_files) - 10} more")

Found 192 videos
  - video_001.mp4
  - video_002.mp4
  - video_003.mp4
  - video_004.mp4
  - video_005.mp4
  - video_006.mp4
  - video_007.mp4
  - video_008.mp4
  - video_009.mp4
  - video_010.mp4
  ... and 182 more


## Process Videos

This will:
1. Test 12 encoding combinations per video (4 CRF × 3 presets)
2. Extract video features
3. Find optimal settings

**Time estimate:** 2-5 minutes per video (CPU encoding)

In [14]:
import warnings
warnings.filterwarnings('ignore')

training_data = []
total_videos = len(video_files)

if total_videos == 0:
    print("❌ No videos to process! Add videos to the folder first.")
else:
    print(f"\n{'='*60}")
    print(f"Starting processing of {total_videos} videos")
    print(f"Estimated time: {total_videos * 2} - {total_videos * 5} minutes (CPU encoding)")
    print(f"{'='*60}\n")

    for idx, video_path in enumerate(video_files, 1):
        try:
            print(f"\n[{idx}/{total_videos}] {video_path.name}")
            
            # Find optimal CRF and preset
            optimal_crf, optimal_preset, quality, size_mb = find_optimal_settings(str(video_path))
            
            # Extract features
            features = extract_features(str(video_path))
            
            # Combine all data
            row = {
                'video_name': video_path.name,
                'optimal_crf': optimal_crf,
                'optimal_preset': optimal_preset,
                'quality_score': quality,
                'file_size_mb': size_mb,
                **features
            }
            training_data.append(row)
            
        except Exception as e:
            print(f"❌ Error processing {video_path.name}: {e}")
            continue

    print(f"\n{'='*60}")
    print(f"✅ Completed! Processed {len(training_data)} / {total_videos} videos")
    print(f"{'='*60}")


Starting processing of 192 videos
Estimated time: 384 - 960 minutes (CPU encoding)


[1/192] video_001.mp4

🎬 Processing: video_001.mp4
  CRF 24 + faster : Quality 96.4, 7.6MB, 6.2s
  CRF 24 + faster : Quality 96.4, 7.6MB, 6.2s
  CRF 24 + fast   : Quality 96.7, 8.2MB, 8.0s
  CRF 24 + fast   : Quality 96.7, 8.2MB, 8.0s
  CRF 24 + medium : Quality 96.6, 7.7MB, 9.4s
  CRF 24 + medium : Quality 96.6, 7.7MB, 9.4s
  CRF 26 + faster : Quality 95.7, 5.9MB, 6.7s
  CRF 26 + faster : Quality 95.7, 5.9MB, 6.7s
  CRF 26 + fast   : Quality 96.0, 6.3MB, 7.9s
  CRF 26 + fast   : Quality 96.0, 6.3MB, 7.9s
  CRF 26 + medium : Quality 96.0, 5.8MB, 8.9s
  CRF 26 + medium : Quality 96.0, 5.8MB, 8.9s
  CRF 28 + faster : Quality 94.9, 4.6MB, 6.0s
  CRF 28 + faster : Quality 94.9, 4.6MB, 6.0s
  CRF 28 + fast   : Quality 95.2, 4.9MB, 7.3s
  CRF 28 + fast   : Quality 95.2, 4.9MB, 7.3s
  CRF 28 + medium : Quality 95.2, 4.5MB, 8.3s
  CRF 28 + medium : Quality 95.2, 4.5MB, 8.3s
  CRF 30 + faster : Quality 93.9, 3

## Save Results to CSV

In [15]:
if training_data:
    # Convert to DataFrame
    df = pd.DataFrame(training_data)

    # Reorder columns
    columns_order = [
        'video_name', 'optimal_crf', 'optimal_preset', 'quality_score', 'file_size_mb',
        'edge_density', 'motion_score', 'brightness', 'color_variance', 
        'resolution', 'fps', 'duration'
    ]
    df = df[columns_order]

    # Save to CSV
    output_file = 'training_data_local.csv'
    df.to_csv(output_file, index=False)

    print(f"\n✅ Saved training data to: {output_file}")
    print(f"\nDataset shape: {df.shape[0]} videos × {df.shape[1]} features")
    print(f"\nFirst 5 rows:")
    display(df.head())

    print("\n📊 Statistics:")
    print(df[['optimal_crf', 'quality_score', 'file_size_mb']].describe())

    print("\n🎯 Preset distribution:")
    print(df['optimal_preset'].value_counts())
else:
    print("❌ No data to save. Process some videos first!")


✅ Saved training data to: training_data_local.csv

Dataset shape: 192 videos × 12 features

First 5 rows:


,video_name,optimal_crf,optimal_preset,quality_score,file_size_mb,edge_density,motion_score,brightness,color_variance,resolution,fps,duration
0,video_001.mp4,30,medium,94.2271,3.537961,0.021577,0.050100,0.353984,0.250147,2073600,25.00000,25.32
1,video_002.mp4,30,faster,94.9696,4.165270,0.016058,0.083528,0.612588,0.236589,2073600,25.00000,37.88
2,video_003.mp4,30,medium,94.2271,3.537961,0.021577,0.050100,0.353984,0.250147,2073600,25.00000,25.32
3,video_004.mp4,30,faster,90.4960,12.172397,0.173314,0.106780,0.476994,0.254383,2073600,29.97003,30.03
4,video_005.mp4,30,faster,95.5579,1.606297,0.008162,0.056225,0.233900,0.175995,983520,25.00000,38.44



📊 Statistics:
       optimal_crf  quality_score  file_size_mb
count   192.000000     192.000000    192.000000
mean     28.895833      92.957672     10.476306
std       1.671938       2.308511     10.772673
min      24.000000      90.007900      0.609561
25%      28.000000      90.894400      3.178821
50%      30.000000      92.224300      6.488330
75%      30.000000      94.950025     14.610506
max      30.000000      97.771600     55.325655

🎯 Preset distribution:
optimal_preset
faster    96
medium    70
fast      26
Name: count, dtype: int64


## Train ML Models

Train RandomForest models to predict optimal CRF and preset based on video features:
- **CRF Model**: Predicts optimal CRF value (24, 26, 28, or 30)
- **Preset Model**: Predicts optimal preset (faster, fast, or medium)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
import joblib

# Load the training data
df = pd.read_csv('training_data_local.csv')

print(f"📊 Loaded dataset: {df.shape[0]} videos × {df.shape[1]} features")
print(f"\n🎯 Target distributions:")
print(f"\nCRF distribution:")
print(df['optimal_crf'].value_counts().sort_index())
print(f"\nPreset distribution:")
print(df['optimal_preset'].value_counts())

# Prepare features and targets
feature_columns = ['edge_density', 'motion_score', 'brightness', 'color_variance', 'resolution', 'fps', 'duration']
X = df[feature_columns]
y_crf = df['optimal_crf']
y_preset = df['optimal_preset']

# Split data (80% train, 20% test)
X_train, X_test, y_crf_train, y_crf_test = train_test_split(X, y_crf, test_size=0.2, random_state=42)
_, _, y_preset_train, y_preset_test = train_test_split(X, y_preset, test_size=0.2, random_state=42)

print(f"\n📚 Training set: {X_train.shape[0]} videos")
print(f"🧪 Test set: {X_test.shape[0]} videos")

# Train CRF model
print(f"\n{'='*60}")
print("Training CRF prediction model...")
print(f"{'='*60}")

crf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

crf_model.fit(X_train, y_crf_train)
y_crf_pred = crf_model.predict(X_test)

crf_accuracy = accuracy_score(y_crf_test, y_crf_pred)
crf_mae = mean_absolute_error(y_crf_test, y_crf_pred)

print(f"\n✅ CRF Model Results:")
print(f"   Accuracy: {crf_accuracy:.2%}")
print(f"   Mean Absolute Error: {crf_mae:.2f}")
print(f"\n   Classification Report:")
print(classification_report(y_crf_test, y_crf_pred))

print(f"\n📊 Feature Importance (CRF Model):")
for feature, importance in sorted(zip(feature_columns, crf_model.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"   {feature:20s}: {importance:.4f}")

# Train Preset model
print(f"\n{'='*60}")
print("Training Preset prediction model...")
print(f"{'='*60}")

preset_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

preset_model.fit(X_train, y_preset_train)
y_preset_pred = preset_model.predict(X_test)

preset_accuracy = accuracy_score(y_preset_test, y_preset_pred)

print(f"\n✅ Preset Model Results:")
print(f"   Accuracy: {preset_accuracy:.2%}")
print(f"\n   Classification Report:")
print(classification_report(y_preset_test, y_preset_pred))

print(f"\n📊 Feature Importance (Preset Model):")
for feature, importance in sorted(zip(feature_columns, preset_model.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"   {feature:20s}: {importance:.4f}")

# Combined accuracy (both CRF and preset correct)
combined_correct = (y_crf_pred == y_crf_test.values) & (y_preset_pred == y_preset_test.values)
combined_accuracy = combined_correct.sum() / len(y_crf_test)

print(f"\n🎯 Combined Accuracy (both CRF and Preset correct): {combined_accuracy:.2%}")

# Save models
joblib.dump(crf_model, 'crf_model.pkl')
joblib.dump(preset_model, 'preset_model.pkl')

print(f"\n{'='*60}")
print(f"✅ Models saved successfully!")
print(f"   - crf_model.pkl")
print(f"   - preset_model.pkl")
print(f"{'='*60}")


📊 Loaded dataset: 192 videos × 12 features

🎯 Target distributions:

CRF distribution:
optimal_crf
24      6
26     25
28     38
30    123
Name: count, dtype: int64

Preset distribution:
optimal_preset
faster    96
medium    70
fast      26
Name: count, dtype: int64

📚 Training set: 153 videos
🧪 Test set: 39 videos

Training combined CRF + Preset prediction model...

✅ CRF Predictions:
   Accuracy: 69.23%
   Mean Absolute Error: 0.77

   Classification Report:
              precision    recall  f1-score   support

          24       1.00      0.33      0.50         3
          26       0.29      1.00      0.44         2
          28       0.86      0.43      0.57        14
          30       0.75      0.90      0.82        20

    accuracy                           0.69        39
   macro avg       0.72      0.67      0.58        39
weighted avg       0.78      0.69      0.69        39


✅ Preset Predictions:
   Accuracy: 66.67%

   Classification Report:
              precision    rec

## Test the Trained Models

Let's test the models on a new video to see predictions:


In [ ]:
# Load models
crf_model = joblib.load('crf_model.pkl')
preset_model = joblib.load('preset_model.pkl')

# Test on a random video from test set
test_idx = 13
test_video = X_test.iloc[test_idx]

# Make predictions
predicted_crf = crf_model.predict([test_video])[0]
predicted_preset = preset_model.predict([test_video])[0]

# Get actual values
actual_crf = y_crf_test.iloc[test_idx]
actual_preset = y_preset_test.iloc[test_idx]

print(f"🎬 Test Video Features:")
print(f"   Edge Density: {test_video['edge_density']:.4f}")
print(f"   Motion Score: {test_video['motion_score']:.4f}")
print(f"   Brightness: {test_video['brightness']:.4f}")
print(f"   Color Variance: {test_video['color_variance']:.4f}")
print(f"   Resolution: {test_video['resolution']:.0f}")
print(f"   FPS: {test_video['fps']:.1f}")
print(f"   Duration: {test_video['duration']:.1f}s")

print(f"\n🎯 Predictions:")
print(f"   Predicted CRF: {predicted_crf} | Actual CRF: {actual_crf} {'✅' if predicted_crf == actual_crf else '❌'}")
print(f"   Predicted Preset: {predicted_preset} | Actual Preset: {actual_preset} {'✅' if predicted_preset == actual_preset else '❌'}")

both_correct = (predicted_crf == actual_crf) and (predicted_preset == actual_preset)
print(f"\n   Both predictions correct: {'✅ YES!' if both_correct else '❌ No'}")

# Get prediction probabilities
crf_probs = crf_model.predict_proba([test_video])[0]
preset_probs = preset_model.predict_proba([test_video])[0]

print(f"\n📊 Prediction Confidence:")
print(f"   CRF probabilities:")
for crf_val, prob in zip(crf_model.classes_, crf_probs):
    print(f"      CRF {int(crf_val)}: {prob:.1%}")
    
print(f"   Preset probabilities:")
for preset_val, prob in zip(preset_model.classes_, preset_probs):
    print(f"      {preset_val:7s}: {prob:.1%}")


🎬 Test Video Features:
   Edge Density: 0.1496
   Motion Score: 0.0684
   Brightness: 0.3374
   Color Variance: 0.2166
   Resolution: 2073600
   FPS: 30.0
   Duration: 31.7s

🎯 Predictions:
   Predicted CRF: 28 | Actual CRF: 28 ✅
   Predicted Preset: faster | Actual Preset: fast ❌

   Both predictions correct: ❌ No

📊 Prediction Confidence:
   CRF probabilities:
      CRF 24: 1.4%
      CRF 26: 22.2%
      CRF 28: 46.3%
      CRF 30: 30.1%
   Preset probabilities:
      faster : 62.4%
      fast   : 26.8%
      medium : 10.8%


## Quick Test with 1 Video

If you just want to test with one video first:

In [16]:
# Test with first video only
if video_files:
    test_video = video_files[0]
    print(f"Testing with: {test_video.name}\n")
    
    try:
        # Quick test - only 3 combinations instead of 12
        crf, preset, quality, size = find_optimal_settings(
            str(test_video),
            crf_values=[26, 28],  # Only test 2 CRF values
            presets=['fast'],      # Only test 1 preset
            quality_threshold=70
        )
        
        features = extract_features(str(test_video))
        
        print(f"\n✅ Test successful!")
        print(f"\nOptimal settings: CRF {crf}, Preset {preset}")
        print(f"Quality: {quality:.1f}, Size: {size:.1f}MB")
        print(f"\nFeatures: {features}")
        
    except Exception as e:
        print(f"❌ Test failed: {e}")
else:
    print("No videos to test!")

Testing with: video_001.mp4


🎬 Processing: video_001.mp4
  CRF 26 + fast   : Quality 96.0, 6.3MB, 7.8s
  CRF 26 + fast   : Quality 96.0, 6.3MB, 7.8s
  CRF 28 + fast   : Quality 95.2, 4.9MB, 7.3s
  ✅ Optimal: CRF 28 + fast (Quality 95.2, 4.9MB)
  CRF 28 + fast   : Quality 95.2, 4.9MB, 7.3s
  ✅ Optimal: CRF 28 + fast (Quality 95.2, 4.9MB)

✅ Test successful!

Optimal settings: CRF 28, Preset fast
Quality: 95.2, Size: 4.9MB

Features: {'edge_density': np.float64(0.0215765595322887), 'motion_score': np.float64(0.05009951941115953), 'brightness': np.float64(0.35398407051863945), 'color_variance': np.float64(0.2501471801154926), 'resolution': 2073600, 'fps': 25.0, 'duration': 25.32}

✅ Test successful!

Optimal settings: CRF 28, Preset fast
Quality: 95.2, Size: 4.9MB

Features: {'edge_density': np.float64(0.0215765595322887), 'motion_score': np.float64(0.05009951941115953), 'brightness': np.float64(0.35398407051863945), 'color_variance': np.float64(0.2501471801154926), 'resolution': 2073600